In [ ]:
import sys
import transformers
if not hasattr(transformers, 'is_torch_npu_available'):
    transformers.is_torch_npu_available = lambda: False

# Fix transformers.utils
import transformers.utils as utils
if not hasattr(utils, 'is_torch_npu_available'):
    utils.is_torch_npu_available = lambda: False

# Fix import_utils
import transformers.utils.import_utils as import_utils

# Add all potentially missing functions
missing_functions = {
    'is_torch_npu_available': lambda: False,
    'is_torch_mps_available': lambda: False,
    'is_torch_xpu_available': lambda: False,
    'is_torch_tpu_available': lambda: False,
    'is_nltk_available': lambda: False,
    'is_torch_neuroncore_available': lambda: False,
    'is_torch_fx_available': lambda: False,
}

for func_name, func in missing_functions.items():
    if not hasattr(import_utils, func_name):
        setattr(import_utils, func_name, func)
    if not hasattr(transformers, func_name):
        setattr(transformers, func_name, func)

if not hasattr(import_utils, 'NLTK_IMPORT_ERROR'):
    import_utils.NLTK_IMPORT_ERROR = "NLTK not available"
    print("✓ Added NLTK_IMPORT_ERROR")

if not hasattr(import_utils, 'is_nltk_available'):
    import_utils.is_nltk_available = lambda: False
    print("✓ Added is_nltk_available")
import pandas as pd
import numpy as np
import os
import copy
from datasets import load_dataset
from pprint import pprint
import re
import spacy
import nltk
from sklearn.utils import shuffle
import argparse
from itertools import chain
import torch
from torch.utils.data import Dataset, DataLoader
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from nltk.corpus import stopwords
import copy
from operator import itemgetter
from tqdm import tqdm
stop_words = stopwords.words('english')
nlp = spacy.load("en_core_web_sm")
from transformers import T5ForConditionalGeneration,T5Tokenizer 
import PyPDF2
import traceback
from typing import Tuple, Union, List, Optional
import gc
from argparse import Namespace
import coreferee
nlp.add_pipe("coreferee")
from collections import defaultdict, Counter
from pathlib import Path
from huggingface_hub import snapshot_download
print("✓ snapshot_download imported successfully")

# Check if PEFT is trying to import it wrong
try:
    import peft
    print(f"PEFT version: {peft.__version__}")
except ImportError as e:
    print(f"PEFT import error: {e}")
from sentence_transformers import SentenceTransformer, util
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)

In [ ]:
import pytorch_lightning as pl
import textwrap
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
pl.seed_everything(42)

In [ ]:
def extract_pdf(pdf_path):
    print("pdf_path:", pdf_path)
    text = ""
    try:
        reader = PyPDF2.PdfReader(pdf_path)
        num_pages = len(reader.pages)
        print("Number of pages",num_pages)
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text += page.extract_text()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text

In [ ]:
base_path = os.getcwd() + "/inputText/request/"
workdir=os.listdir(base_path)
if '.DS_Store' in workdir:
  workdir.remove('.DS_Store')
print (base_path)
# List of files in above directory, making sure no directories are included
#files = (file for file in os.listdir(text_path) if os.path.isfile(os.path.join(text_path, file)))
def get_files(path):
    file_names = [file_name for file_name in os.listdir(base_path) if os.path.isfile(os.path.join(base_path, file_name))]
    file_paths = [os.path.join(base_path, file_name) for file_name in file_names]
    return file_paths

file_paths=get_files(base_path)

for file_path in file_paths:
    if file_path.endswith('.pdf'):
        key_name = Path(file_path).stem
        filePath= os.path.join(base_path,key_name)
        outFile = os.path.join(base_path,f"{key_name}.txt")
        # outFile = ''.join(base_path,f"/{key_name}.txt")
        print(f"Processing {filePath}...")
        with open(file_path, 'rb') as f:
            text = extract_pdf(f)
            with open(outFile, "w", encoding="utf-8") as extract:
                extract.write(text)
            print(f"Extracted text from {file_path} and saved to {outFile}")

    elif file_path.endswith('.txt'):
        outFile = file_path

In [ ]:
# def readTextFile(path):
#     outText = []
#     with open(outFile, "r") as file:
#         # mypdf =json.load(file)
#         T = mypdf.readlines(file)
#         final = [i.replace('\r\n',' ').replace('b\r\n', '').replace('\n', '')  for i in T]
#         outText.append(T)
#     sentText= extract_sentences(outText)
#     final = [i.replace("\n'"," ") for i in sentText]
#     #result = re.sub(r'http\S+', '', str(final))
#     newFinal =[re.sub("[\(%\[].*?[\)\n]]", " ", i) for i in final]
#     newFinal =[re.sub(r'http\S+', '', i) for i in newFinal]
#     df2 =pd.DataFrame(final, columns= ['sentences'])
#     df3 =pd.DataFrame(newFinal, columns= ['sentences'])

#     for i in range(0, len(df2)):
#         df2.at[i, 'sentences_new'] = re.sub(r'\b(\d+\w{1,2})\b'," ",str(df2.loc[i,'sentences']))
#         df2.at[i, 'sentences_new'] = re.sub(r'[\d+]'," ",str(df2.loc[i,'sentences_new']))
#         df2.at[i, 'sentences_new'] = re.sub(r'[^\w\s]'," ",str(df2.loc[i,'sentences_new']))
#         #df2.at[i, 'sentences_new'] = re.sub(r'\b\w{1}\b'," ",str(df2.loc[i,'sentences_new']))
#         df2['text'] = df2['sentences_new'].str.replace(r"\s*https?:'//\S+(\s+|$)", " ").str.strip()
#         df2['text2'] = df2['text'].str.replace(r"ufeff", " ").str.strip()
#     return df2

def readTextFile(path):   
    outText = []
    for file_path in path:
        if file_path.endswith('.txt'):
                try:
                    txtFile= open(file_path, "r")
                    mypdf = txtFile.readlines()
                    outText.append(mypdf)
                except FileNotFoundError:
                    print("File not found.")
                    print(traceback.format_exc())
                except Exception as e:
                    print("Error:", e)
                finally:
                    if 'txtFile' in locals():
                        txtFile.close()

    print("raw text has been read")
    return outText

    

## updated sentence extraction to include co-reference resolution

In [ ]:
# def extract_sentences(texts):
#         content = nlp(str(texts))
#         content_resolved = content._.coref_resolved
#         # extract sentences from the resolved content
#         doc= nlp(content_resolved)
#         sentences=[x.text for x in doc.sents]
#         return sentences

# # formatting and cleaning text
# def format(outText):
#         """Fix syntax issues in incoming text fields, before any further processing
#         """
#         extracted_sentences= extract_sentences(outText)
#         texts= ' '.join(extracted_sentences)
#         print("sample sentences", extracted_sentences[:10])
#         return texts

In [ ]:
# update for the above function
from collections import defaultdict, Counter

class SentenceProcessor:
    """Process text for entity extraction and question generation"""
    
    def __init__(self):
        self.nlp = nlp
        self.entities = set()
        self.entity_mentions = defaultdict(list)
    
    def process_text(self, text):
        """Main processing function that returns resolved text and entities"""
        
        doc = self.nlp(text)
        
        # Step 1: Resolve coreferences
        resolved_text = self.get_coreferences(doc)
        
        # Step 2: Process resolved text to get clean sentences
        resolved_doc = self.nlp(resolved_text)
        
        # Step 3: Extract entities from resolved text
        corpus_entities = self.entity_extraction(resolved_doc)
        
        # Step 4: Get sentences for question generation
        sentences = self.get_sentences(resolved_doc)
        print("sentences type",type(sentences))
        print("corpus_entities type",type(corpus_entities))

        return sentences, corpus_entities
        # return {
        #     'sentences': sentences,
        #     'corpus_entities': corpus_entities
        # }
    
    def get_coreferences(self, doc):
        """Resolve all coreferences in the texts"""
        
        chains = doc._.coref_chains
        
        if not chains:
            return doc.text
        
        # Build token replacement map
        token_replacements = {}
        
        for chain in chains:
            # Find the most descriptive mention (usually the first noun phrase)
            main_mention = self.find_ent_ref(chain)
            main_text = " ".join([token.text for token in main_mention])
            
            # Map all other mentions to the main mention
            for mention in chain:
                if mention != main_mention:
                    # Store the span to be replaced
                    start_token = mention[0].i
                    end_token = mention[-1].i
                    token_replacements[(start_token, end_token)] = main_text
        
        # Reconstruct text with replacements
        resolved_tokens = []
        i = 0
        
        while i < len(doc):
            replaced = False
            
            # Check if current position starts a mention to be replaced
            for (start, end), replacement in token_replacements.items():
                if i == start:
                    resolved_tokens.append(replacement)
                    # Add spacing if needed
                    if end + 1 < len(doc) and not doc[end + 1].is_punct:
                        resolved_tokens.append(" ")
                    i = end + 1
                    replaced = True
                    break
            
            if not replaced:
                # Keep original token with its spacing
                resolved_tokens.append(doc[i].text_with_ws)
                i += 1
        
        resolved_text = "".join(resolved_tokens)
        
        # Clean up any double spaces
        resolved_text = re.sub(r'\s+', ' ', resolved_text)
        resolved_text = re.sub(r'\s+([.,!?;:])', r'\1', resolved_text)
        
        return resolved_text
    
    def find_ent_ref(self, chain):
        """Find the most descriptive mention in a coreference chain"""
        
        best_reference = chain[0]
        best_score = 0
        
        for ref in chain:
            score = 0
            # Prefer non-pronouns
            if not all(token.pos_ == "PRON" for token in ref):
                score += 10
            if any(token.pos_ == "PROPN" for token in ref) or any(token.ent_type_ for token in ref):
                 score += 5
            
            if score > best_score:
                best_score = score
                best_reference = ref
        
        return best_reference
    
    def entity_extraction(self, doc):
        """Extract all unique entities from resolved text"""
        
        named_entities = {
            'PERSON': set(),
            'ORG': set(),
            'GPE': set(),  
            'DATE': set(),
            'TIME': set(),
            'MONEY': set(),
            'PERCENT': set(),
            'FACILITY': set(),
            'LOC': set(),
            'PRODUCT': set(),
            'EVENT': set(),
            'WORK_OF_ART': set(),
            'LAW': set(),
            'LANGUAGE': set(),
            'QUANTITY': set(),
            'ORDINAL': set(),
            'CARDINAL': set(),
            'MISC': set()  # Miscellaneous entities
        }
        
        # Extract named entities
        for ent in doc.ents:
            if ent.label_ in named_entities:
                named_entities[ent.label_].add(ent.text)
            else:
                named_entities['MISC'].add(ent.text)
        
        # Also extract noun phrases that might be important entities
        noun_phrases = set()
        for chunk in doc.noun_chunks:
            # Filter out pronouns and very common words
            if (not all(token.pos_ == "PRON" for token in chunk) and 
                len(chunk.text) > 2 and 
                chunk.text.lower() not in ['the', 'a', 'an']):
                noun_phrases.add(chunk.text)
        
        named_entities['NOUN_PHRASES'] = noun_phrases
        
        # Remove empty categories
        named_entities = {k: v for k, v in named_entities.items() if v}
        
        # Create a summary of all unique entities
        unique_entities = set()
        for entity_list in named_entities.values():
            unique_entities.update(entity_list)
        
        return {
            'by_type': named_entities,
            'all_unique': sorted(unique_entities)
        }
    
    def get_sentences(self, doc):
        """Prepare sentences for question generation
        use scores to prioritize sentences with useful information.
        """
        
        sentences = []
        
        for sent_idx, sent in enumerate(doc.sents):
            # Extract sentence information
            sentence_entities = [(ent.text, ent.label_) for ent in sent.ents]
            
            # Calculate sentence quality score for Quality assessment
            assessment_score = 0
            
            # Prefer sentences with entities
            assessment_score += len(sentence_entities) * 2
            
            # # Prefer sentences with facts (dates, numbers, etc.)
            if any(ent[1] in ['DATE', 'TIME', 'MONEY', 'PERCENT', 'QUANTITY'] 
                   for ent in sentence_entities):
                assessment_score += 3
            
            # Prefer sentences with proper nouns
            if any(token.pos_ == "PROPN" for token in sent):
                assessment_score += 2
            
            # Prefer sentences of moderate length (not too short, not too long)
            word_count = len([token for token in sent if not token.is_punct])
            if 5 < word_count < 30:
                assessment_score += 2
            
            sentence_info = {
                'text': sent.text,
                'index': sent_idx,
                'entities': sentence_entities,
                'word_count': word_count,
                'qa_score': assessment_score,
                'best_by_assessment': assessment_score >= 3,
                'tokens': [token.text for token in sent]
            }
            
            sentences.append(sentence_info)
        
        # Sort by Quality score for prioritization
        sentences.sort(key=lambda x: x['qa_score'], reverse=True)
        print("sentence should be a dictionary",type(sentences))
        return sentences
    
    def get_coref_info(self, doc):
        """Get coreference chain information"""
        
        chains = doc._.coref_chains
        if not chains:
            return []
        
        coref_info = []
        for chain_idx, chain in enumerate(chains):
            mentions = []
            for mention in chain:
                mention_text = " ".join([token.text for token in mention])
                mention_type = "pronoun" if all(token.pos_ == "PRON" for token in mention) else "noun"
                mentions.append({
                    'text': mention_text,
                    'type': mention_type,
                    'start': mention[0].idx,
                    'end': mention[-1].idx + len(mention[-1].text)
                })
            
            coref_info.append({
                'chain_id': chain_idx,
                'mentions': mentions,
                'main_entity': mentions[0]['text'] if mentions else None
            })
        
        return coref_info



## updated text preprocessor compartible with most text and technical documents after pdf extraction to txt files

In [ ]:
#!/usr/bin/env python3

class TextPreprocessor:
    
    def __init__(self):
        """Initialize the preprocessor with common patterns."""
        # Patterns for different types of numbering
        self.numbering_patterns = [
            # Section numbers like 1.1, 1.2.3, etc.
            r'^\s*\d+(?:\.\d+)*\s+',
            # Simple line numbers at start
            r'^\s*\d+\s+',
            # Parenthetical numbers like (1), (2), etc.
            r'^\s*\(\d+\)\s*',
            # Letter numbering like a), b), etc.
            r'^\s*[a-zA-Z]\)\s*',
            # Roman numerals
            r'^\s*[IVXLCDM]+\.\s*',
            r'^\s*[ivxlcdm]+\.\s*',
            # Bullet points with numbers
            r'^\s*[-•*]\s*\d+\.\s*',
            # Sub-section numbers like 5.14.2
            r'^Sub-Section\s+\d+(?:\.\d+)*\s*',
        ]
        
        # Document identifiers and headers
        self.document_patterns = [
            r'IW-CDS-\d+-\d+\s*\([^)]+\)',  
            r'^\s*\d+\s*$',  
        ]
        
    def remove_numbering(self, text: str) -> str:
        """
        Remove various types of numbering from the text.
        """
        # lines = text.split('\n')
        lines = text
        cleaned_lines = []
        
        for line in lines:
            # Skip empty lines
            if not line.strip():
                cleaned_lines.append('')
                continue
                
            # Remove document identifiers if they're standalone
            skip_line = False
            for pattern in self.document_patterns:
                if re.match(pattern + r'\s*$', line):
                    skip_line = True
                    break
            
            if skip_line:
                continue
            
            # Remove numbering patterns from the beginning of lines
            cleaned_line = line
            for pattern in self.numbering_patterns:
                cleaned_line = re.sub(pattern, '', cleaned_line)
            
            # Remove inline section references like "Section 5.14"
            cleaned_line = re.sub(r'\bSection\s+\d+(?:\.\d+)*\b', 'Section', cleaned_line)
            
            # Remove inline sub-section references
            cleaned_line = re.sub(r'\bSub-Section\s+\d+(?:\.\d+)*\b', 'Sub-Section', cleaned_line)
            
            cleaned_lines.append(cleaned_line)
        
        return '\n'.join(cleaned_lines)
    
    def remove_page_headers_footers(self, text: str) -> str:
        """
        Remove common page headers and footers.
        """
        lines = text.split('\n')
        # lines = text
        cleaned_lines = []
        
        for i, line in enumerate(lines):
            # Skip document ID lines that appear as headers/footers
            if re.search(r'IW-CDS-\d+-\d+.*Revision.*\d{4}', line):
                continue
            
            # Skip standalone page numbers
            if re.match(r'^\s*\d{1,4}\s*$', line):
                # Check if it's likely a page number (isolated number line)
                if i > 0 and i < len(lines) - 1:
                    prev_empty = not lines[i-1].strip()
                    next_empty = not lines[i+1].strip()
                    if prev_empty or next_empty:
                        continue
            
            cleaned_lines.append(line)
        
        return '\n'.join(cleaned_lines)
    
    def normalize_whitespace(self, text: str) -> str:
        """
        Normalize whitespace in the text.
        """
        # Replace multiple spaces with single space
        text = re.sub(r' +', ' ', text)
        
        # Replace multiple newlines with maximum of two
        text = re.sub(r'\n{3,}', '\n\n', text)
        
        # Remove trailing whitespace from lines
        lines = text.split('\n')
        # lines = text
        lines = [line.rstrip() for line in lines]
        
        return '\n'.join(lines)
    
    def fix_word_breaks(self, text: str) -> str:
        """
        Fix words that are broken across lines with hyphens.
        """
        # Fix words broken with hyphens at line ends
        text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
        
        return text
    
    def clean_special_characters(self, text: str, preserve_punctuation: bool = True) -> str:
        """
        Clean special characters while preserving readability.
        """
        # Replace common special characters
        replacements = {
            '"': '"',
            '"': '"',
            ''': "'",
            ''': "'",
            '–': '-',
            '—': '-',
            '…': '...',
            '\u00a0': ' ',  # Non-breaking space
            '\u200b': '',   # Zero-width space
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Remove control characters but preserve newlines and tabs
        text = ''.join(char for char in text if char in '\n\t' or not ord(char) < 32)
        
        return text
    
    def merge_broken_sentences(self, text: str) -> str:
        """
        Merge sentences that are incorrectly broken across lines.
        """
        lines = text.split('\n')
        merged_lines = []
        
        i = 0
        while i < len(lines):
            current_line = lines[i].strip()
            
            if not current_line:
                merged_lines.append('')
                i += 1
                continue
            
            # Check if current line doesn't end with sentence-ending punctuation
            # and the next line doesn't start with capital letter (likely continuation)
            if (i < len(lines) - 1 and 
                current_line and 
                not current_line[-1] in '.!?:;' and
                lines[i + 1].strip() and 
                not re.match(r'^[A-Z\d\(\[]', lines[i + 1].strip())):
                
                # Merge with next line
                next_line = lines[i + 1].strip()
                merged_lines.append(current_line + ' ' + next_line)
                i += 2
            else:
                merged_lines.append(current_line)
                i += 1
        
        return '\n'.join(merged_lines)
    
    def extract_sections(self, text: str) -> dict:
        """
        Extract sections from the document based on headers.
        """
        sections = {}
        current_section = "Introduction"
        current_content = []
        
        lines = text.split('\n')
        
        for line in lines:
            # Check if line is a section header (usually in title case or all caps)
            if (len(line.strip()) > 0 and 
                len(line.strip()) < 100 and
                (line.isupper() or 
                 re.match(r'^[A-Z][A-Za-z\s]+$', line.strip()) or
                 'Section' in line)):
                
                # Save previous section
                if current_content:
                    sections[current_section] = '\n'.join(current_content).strip()
                
                current_section = line.strip()
                current_content = []
            else:
                current_content.append(line)
        
        # Save last section
        if current_content:
            sections[current_section] = '\n'.join(current_content).strip()
        
        return sections
    
    def preprocess(self, 
                  text: str,
                  remove_numbers: bool = True,
                  remove_headers: bool = True,
                  normalize_spaces: bool = True,
                  fix_breaks: bool = True,
                  clean_chars: bool = True,
                  merge_sentences: bool = False) -> str:
        """
        Apply all preprocessing steps to the text.
            text: Input text
            remove_numbers: Remove numbering
            remove_headers: Remove page headers/footers
            normalize_spaces: Normalize whitespace
            fix_breaks: Fix word breaks
            clean_chars: Clean special characters
            merge_sentences: Merge broken sentences
        """
        if remove_numbers:
            text = self.remove_numbering(text)
        
        if remove_headers:
            text = self.remove_page_headers_footers(text)
        
        if fix_breaks:
            text = self.fix_word_breaks(text)
        
        if clean_chars:
            text = self.clean_special_characters(text)
        if merge_sentences:
            text = self.merge_broken_sentences(text)
        if normalize_spaces:
            text = self.normalize_whitespace(text)
        return text
    
    def process_file(self, 
                    text: str,
                    output_path: Optional[str] = None,
                    **kwargs) -> str:
        """
        Process text file.
        """
        # Read input file
        # with open(input_path, 'r', encoding='utf-8', errors='ignore') as f:
        #     text = f.read()
        output_path = os.getcwd() + "/output/processed_text.txt"
        # Process text
        processed_text = self.preprocess(text, **kwargs)
        df= pd.DataFrame([processed_text], columns=['sentences'])
        # Save if output path provided
        if output_path:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(df)
            print(f"Processed text saved to: {output_path}")
        
        return processed_text
    



In [ ]:
#Load base model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
print("Downloading SQuAD dataset...")
dataset = load_dataset('squad')
print("Dataset structure:")
print(dataset)
print("\nFirst training example:")
print(dataset['train'][0])

# Process training data with correct structure
print("\nProcessing training data...")
train_data = []

for item in dataset['train']:
    context = item['context']
    question = item['question']  # Single question, not a list
    answers = item['answers']     # Dictionary with 'text' and 'answer_start'
    
    # Get the first answer (SQuAD has multiple possible answers)
    answer_text = answers['text'][0] if answers['text'] else ""
    answer_start = answers['answer_start'][0] if answers['answer_start'] else 0
    
    train_data.append({
        'context': context,
        'question': question,
        'answer': answer_text,
        'answer_start': answer_start
    })

# Process validation data
print("Processing validation data...")
val_data = []

for item in dataset['validation']:
    context = item['context']
    question = item['question']
    answers = item['answers']
    
    answer_text = answers['text'][0] if answers['text'] else ""
    answer_start = answers['answer_start'][0] if answers['answer_start'] else 0
    
    val_data.append({
        'context': context,
        'question': question,
        'answer': answer_text,
        'answer_start': answer_start
    })

# Convert to DataFrames
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)

print(f"\nDataset sizes:")
print(f"Train samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
os.makedirs('model/t5/dataset', exist_ok=True)
train_df.to_csv(os.getcwd() + '/model/t5/dataset/squad_t5_train.csv', index=False)
val_df.to_csv(os.getcwd() + '/model/t5/dataset/squad_t5_val.csv', index=False)

In [ ]:
#load stored trained squad dataset
train_save_path = os.getcwd() + '/model/t5/dataset/squad_t5_train.csv'
validation_save_path = os.getcwd() + '/model/t5/dataset/squad_t5_val.csv'

In [ ]:
#load default dataset and splits for pretraining- already saved in folder
train_data = load_dataset('squad', split='train')
validate_data = load_dataset('squad', split='validation')
sample_validation_dataset = next(iter(validate_data))
df_train = pd.DataFrame(train_data, columns = ['context','answers', 'question'])

counter_train= 0
for index,val in enumerate(tqdm(train_data)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words > 5:
        df_train.loc[counter_train]= [passage] + [answer] + [question]
        counter_train = counter_train + 1

counter_val = 0
df_validation = pd.DataFrame(validate_data,columns = ['context', 'answers', 'question'])
for index,val in enumerate(tqdm(validate_data)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words > 5:
        df_validation.loc[counter_val]= [passage] + [answer] + [question]
        counter_val = counter_val + 1

# save the dataset
df_train.to_csv(train_save_path, index = False)
df_validation.to_csv(validation_save_path, index = False)

In [ ]:
#question generation class

class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_inp=256,max_len_out=64):
        self.path = filepath

        self.passage_column = "context"
        #self.answer = "answer"
        self.question = "question"

        self.data = pd.read_csv(self.path)
        # self.data = pd.read_csv(self.path,nrows=1000)

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.skippedcount =0
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  
        target_mask = self.targets[index]["attention_mask"].squeeze()  

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for idx in tqdm(range(len(self.data))):
            passage = self.data.loc[idx, self.passage_column]
            question = self.data.loc[idx, self.question]
            input_ = f"context: {str(passage)}" 
            target = question

            # remove encoding length of input to ensure token length check 
            test_input_encoding = self.tokenizer(input_,
                                        truncation=False,
                                        return_tensors="pt")

            length_of_input_encoding = len(test_input_encoding['input_ids'][0])


            if length_of_input_encoding > self.max_len_input:
              self.skippedcount = self.skippedcount + 1
              continue

            # tokenize inputs 
            tokenized_inputs = self.tokenizer(
                input_, max_length=self.max_len_input, padding='max_length', truncation=True, return_tensors="pt"
            )
            # tokenize targets
            with self.tokenizer.as_target_tokenizer():
                tokenized_targets = self.tokenizer(
                    target, max_length=self.max_len_output, padding='max_length', truncation=True,return_tensors="pt"
                )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)


In [ ]:
# save validation and train dataset
train_dataset = QuestionGenerationDataset(t5_tokenizer,train_save_path)
validation_dataset = QuestionGenerationDataset(t5_tokenizer,validation_save_path)
# train sample
# train_sample = train_dataset[51]
# decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
# decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

In [ ]:
# class T5FineTuner(pl.LightningModule):
#     def __init__(self,hparams, t5model, t5tokenizer):
#         super(T5FineTuner, self).__init__()
#         self.hparams = hparams
#         self.model = t5model
#         self.tokenizer = t5tokenizer


#     def forward( self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
#          outputs = self.model(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             decoder_attention_mask=decoder_attention_mask,
#             labels=lm_labels,
#         )

#          return outputs


#     def training_step(self, batch, batch_idx):
#         outputs = self.forward(
#             input_ids=batch["source_ids"],
#             attention_mask=batch["source_mask"],
#             decoder_input_ids = batch["target_ids"],
#             decoder_attention_mask=batch['target_mask'],
#             lm_labels=batch['labels']
#         )

#         loss = outputs[0]
#         self.log('train_loss',loss)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         outputs = self.forward(
#             input_ids=batch["source_ids"],
#             attention_mask=batch["source_mask"],
#             decoder_input_ids = batch["target_ids"],
#             decoder_attention_mask=batch['target_mask'],
#             lm_labels=batch['labels']
#         )

#         loss = outputs[0]
#         self.log("val_loss",loss)
#         return loss

#     def train_dataloader(self):
#         return DataLoader(train_dataset, 
#                           batch_size=self.hparams.batch_size,
#                           shuffle=True,
#                           num_workers=0)

#     def val_dataloader(self):
#         return DataLoader(validation_dataset, 
#                           batch_size=self.hparams.batch_size,
#                           shuffle=True,
#                           num_workers=0)


#     # optimizer changed for CPU training to torch.optim.SGD
#     def configure_optimizers(self):
#         optimizer = torch.optim.SGD(self.parameters(), lr=3e-4, eps=1e-8)
#         return optimizer


# args_dict = dict(
#     batch_size=1,
# )

# args = argparse.Namespace(**args_dict)

# model = T5FineTuner(args,t5_model,t5_tokenizer)

# trainer = pl.Trainer(max_epochs = 4, gpus=0,progress_bar_refresh_rate=30)

# trainer.fit(model)

In [ ]:
# LoRA Configuration for CPU
def peft_model(model_name='t5-base'):
    """Create T5 with LoRA for CPU training"""
    
    print("Loading base model...")
    model = T5ForConditionalGeneration.from_pretrained(
        model_name,
        torch_dtype=torch.float32,  # FP32 for CPU
        low_cpu_mem_usage=True
    )
    
    # LoRA configuration - optimized for CPU
    lora_config = LoraConfig(
        r=16,  # Rank - higher = better quality, more memory
        lora_alpha=32,  # Scaling parameter
        target_modules=["q", "v"],  # Which layers to adapt
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.SEQ_2_SEQ_LM,
    )
    
    # Apply LoRA
    model = get_peft_model(model, lora_config)
    
    # Print trainable parameters
    model.print_trainable_parameters()
    # Output: trainable params: 2,359,296 || all params: 223,020,032 || trainable%: 1.06
    
    return model

In [ ]:

class CPUTrainer:
    def __init__(self, model, tokenizer, hparams, train_dataset, validation_dataset):
        self.model = model
        self.tokenizer = tokenizer
        self.hparams = hparams
        self.train_data = train_dataset
        self.val_data = validation_dataset
        self.device = torch.device('cpu')

    def forward( self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None):
         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

         return outputs


    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            labels=batch['labels']
        )
        loss = outputs.loss
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            labels=batch['labels']
        )

        loss = outputs.loss
        return loss

    def train_dataloader(self):
        return DataLoader(self.train_data, 
                          batch_size=self.hparams.batch_size,
                          shuffle=True,
                          num_workers=0)

    def val_dataloader(self):
        return DataLoader(self.val_data, 
                          batch_size=self.hparams.batch_size,
                          shuffle=False,
                          num_workers=0)


    # optimizer changed for CPU training to torch.optim.SGD
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=3e-4)
        return optimizer
    
    def train(self, epochs=3, batch_size=1, learning_rate=1e-3):
        """Custom training loop optimized for CPU"""
        
        # DataLoader with minimal memory footprint
        train_loader = self.train_dataloader()
        
        validate_loader = self.val_dataloader()
            
        optimizer = self.configure_optimizers()
        
            
        self.model.train()
        
        for epoch in range(epochs):
            total_loss = 0
            self.model.train()
            train_progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
           
            for batch_idx, batch in enumerate(train_progress_bar):    
                # Forward pass
                train_outputs = self.training_step(batch, batch_idx)
                total_loss += train_outputs.item()
                # Backward pass
                optimizer.zero_grad()
                train_outputs.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                
                optimizer.step()
                
                # Update progress bar
                train_progress_bar.set_postfix({'loss': train_outputs.item()})
                
                # Aggressive memory cleanup every 10 steps
                if batch_idx % 10 == 0:
                    gc.collect()
            val_loss= 0
            self.model.eval()
            validate_progress_bar = tqdm(validate_loader, desc=f'Epoch {epoch+1}/{epochs}')
            with torch.no_grad():
                for batch_idx, batch in enumerate(validate_progress_bar):  
                    # Forward pass
                    val_outputs = self.validation_step(batch, batch_idx)
                    
                    # Update progress bar
                    validate_progress_bar.set_postfix({'val_loss': val_outputs.item()})
                    
                    # Aggressive memory cleanup every 10 steps
                    if batch_idx % 10 == 0:
                        gc.collect()
                    
            avg_train_loss = total_loss / len(train_loader)
            avg_val_loss = val_loss / len(validate_loader)
            print(f"Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")
            print(f"Epoch {epoch+1} -  Validation Loss: {avg_val_loss:.4f}")
            
            # Save checkpoint
            self.save_checkpoint(epoch)

    def save_checkpoint(self, epoch):
        """Save model checkpoint"""
        self.model.save_pretrained(f'./model/lora_checkpoint_epoch_{epoch}')
        print(f"Checkpoint saved for epoch {epoch+1}")

In [ ]:
# Initialize
tokenizer = T5Tokenizer.from_pretrained('t5-base')
hparams= Namespace(
    epochs=3,
    batch_size=1,  # Keep at 1 for 8GB RAM
    learning_rate=1e-3  # Higher LR works well with LoRA
)
train_dataset = QuestionGenerationDataset(tokenizer,train_save_path)
validation_dataset = QuestionGenerationDataset(tokenizer,validation_save_path)

# Create LoRA model
model = peft_model('t5-base')

# Initialize trainer
trainer = CPUTrainer(model, tokenizer, hparams, train_dataset, validation_dataset)

# Train - adjust parameters based on your RAM
trainer.train(epochs=hparams.epochs)


# Save final model should be saved in the same folder
model.save_pretrained(os.getcwd() + '/model/final_lora_model')
tokenizer.save_pretrained(os.getcwd() + '/model/final_lora_model')
print("Training complete!")

In [ ]:
# Initialize includes  boto3 for S3 upload

tokenizer = T5Tokenizer.from_pretrained('t5-base')
hparams= Namespace(
    epochs=3,
    batch_size=1,  # Keep at 1 for 8GB RAM
    learning_rate=1e-3  # Higher LR works well with LoRA
)
train_dataset = QuestionGenerationDataset(tokenizer,train_save_path)
validation_dataset = QuestionGenerationDataset(tokenizer,validation_save_path)

# Create LoRA model
model = peft_model('t5-base')

# Initialize trainer
trainer = CPUTrainer(model, tokenizer, hparams, train_dataset, validation_dataset)

# Train - adjust parameters based on your RAM
trainer.train(epochs=hparams.epochs)

# Save final model in the same location
local_path = os.getcwd() + '/model/final_lora_model'
model.save_pretrained(local_path)
tokenizer.save_pretrained(local_path)
s3 = boto3.client('s3')
bucket_name = "amzn-agocqs"
for root, dirs, files in os.walk(local_path):
        for file in files:
            local_file = os.path.join(root, file)
            s3_key = f"models/final/{file}"
            
            print(f"Uploading {file}...")
            s3.upload_file(local_file, bucket_name, s3_key)
    
print(f"✓ Model uploaded to s3://{bucket_name}/models/final/")
print("Training complete!")

In [ ]:
# verify AWS account has access to S3 bucket
# import sys
# import subprocess

# def setup_aws_s3():
#     """Setup S3 access in EC2 container"""
    
#     # 1. Install boto3 if needed
#     try:
#         import boto3
#         print("✓ boto3 already installed")
#     except ImportError:
#         print("Installing boto3...")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", "boto3"])
#         import boto3
    
#     # 2. Check AWS credentials
#     # On EC2 with IAM role, this should work automatically
#     try:
#         s3 = boto3.client('s3')
        
#         # Test access
#         response = s3.list_buckets()
#         print(f"✓ AWS access working - found {len(response['Buckets'])} buckets")
        
#         # Check which credential method is being used
#         import boto3.session
#         session = boto3.session.Session()
#         credentials = session.get_credentials()
        
#         if credentials:
#             # Check if using instance role
#             provider = credentials.method
#             print(f"✓ Using credential provider: {provider}")
#             if provider == 'iam-role':
#                 print("  Using EC2 instance IAM role (best practice!)")
        
#     except Exception as e:
#         print(f"✗ AWS credentials not configured: {e}")
#         print("\nOptions to fix:")
#         print("1. Attach IAM role to EC2 instance (recommended)")
#         print("2. Set environment variables: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY")
#         print("3. Mount ~/.aws/credentials in container")
    
#     return boto3

# # Run setup
# boto3 = setup_aws_s3()

In [ ]:

def main():
    # Initialize
    tokenizer = T5Tokenizer.from_pretrained('t5-base')
    hparams= Namespace(
        epochs=3,
        batch_size=1,  # Keep at 1 for 8GB RAM
        learning_rate=1e-3  # Higher LR works well with LoRA
    )
    train_dataset = QuestionGenerationDataset(tokenizer,train_save_path)
    validation_dataset = QuestionGenerationDataset(tokenizer,validation_save_path)

    # Create LoRA model
    model = peft_model('t5-base')
    
    # Initialize trainer
    trainer = CPUTrainer(model, tokenizer, hparams, train_dataset, validation_dataset)
    
    # Train - adjust parameters based on your RAM
    trainer.train(epochs=hparams.epochs)

    # Save final model should be saved in the same folder
    model.save_pretrained(os.getcwd() + '/model/final_lora_model')
    tokenizer.save_pretrained(os.getcwd() + '/model/final_lora_model')
    print("Training complete!")


In [ ]:
# print ("Saving model")
# save_path_model = os.getcwd() + '/model/t5/model/'
# save_path_tokenizer = os.getcwd() + '/model/t5/tokenizer/'
# model.model.save_pretrained(save_path_model)
# t5_tokenizer.save_pretrained(save_path_tokenizer)

In [ ]:
# Load trained model for inference
trained_model_path = os.getcwd() + '/model/final_lora_model' 
trained_tokenizer = os.getcwd() + '/model/final_lora_tokenizer' 
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

base_model = T5ForConditionalGeneration.from_pretrained('t5-base')
model = PeftModel.from_pretrained(base_model, trained_model_path)
#check device cuda availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device", device)
model = model.to(device)

In [ ]:
# Generate questions with our data using trained model
def generate(df2):
    
    context = df2['sentences_new']
    text = "context: "+context
    input_ids =[]
    attention_mask = []
    for x in (text):
      encoding = tokenizer.encode_plus(x,max_length =512, padding=True, return_tensors="pt")
      print (encoding.keys())
      input_ids.append(encoding["input_ids"].to(device))
      attention_mask.append(encoding["attention_mask"].to(device))
      input_ids, attention_mask = encoding["input_ids"].to(device),encoding["attention_mask"].to(device)
    data = dict(zip(input_ids, attention_mask))

    model.eval()
    cqs = []

    for x in (text):
      encoding = tokenizer.encode_plus(x,max_length =512, padding=True, return_tensors="pt")
      print (encoding.keys())
      input_ids, attention_mask = encoding["input_ids"].to(device),encoding["attention_mask"].to(device)
      beam_outputs = model.generate(
          input_ids=input_ids,attention_mask=attention_mask,
          max_length=72,
          early_stopping=True,
          num_beams=5,
          num_return_sequences=3
      )

      for beam_output in beam_outputs:
          sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
          cqs.append(sent)
    return cqs

In [ ]:
# Post processing of generated questions
def postprocess(cqs):
    questions =[]
    with open(os.getcwd() + "/output/CQs.txt", "w") as file:
        file.write(str(cqs))
    with open(os.getcwd() + "/output/CQs.txt","r",encoding="utf-8") as f:
        for line in f.readlines():
            questions.append(line)
    new_questions = []
    for i in (questions):
        x = str(i).split(":")
        x = str(i).replace("question", '')
        new_questions.append(x)
    new_questions2 = str(new_questions).split(",")
    dcqs =pd.DataFrame(new_questions2)
    dcqs.columns = ['CQs']
    dcqs['CQs'] = dcqs['CQs'].map(lambda x: x.replace(":", '').replace("'", '').replace('[', '').replace(']', '').replace('"', ''))
    
    for i in range(0, len(dcqs)):
        dcqs.at[i, 'newCQs'] =re.sub(r"\s\([A-Z][a-z]+,\s[A-Z][a-z]?\.[^\)]*,\s\d{4}\)","",str(dcqs.loc[i,'CQs']))
    with open(os.getcwd() + "/output/cq_output.txt", "w") as f:
        f.write("\n".join(dcqs["newCQs"]))
    return dcqs
    # # dcqs.head()

In [ ]:
#Abstract pattern extraction
def mark_chunk(cq, spans, chunktype, offset, counter):
    for (start, end) in spans:  # for each span of EC/PC candidate
        cq = cq[:start - offset] + chunktype + str(counter) +\
            cq[end - offset:]  # substitute that candidate with EC/PC marker
        offset += (end - start) - len(chunktype) - 1  # check by how much the total length of CQ changed
    return cq, offset


def extract_EC_chunks(cq):
    """
        Find EC chunks and replace their occurences with EC tags
    """

    def _get_EC_span_reject_wh_starters(chunk):
        """
            By default, SpaCy treats question words (wh- pronouns starting questions: where, what,...)
            as nouns, so often if questions starts with wh- pronoun + noun (like: What software is the best?)
            the whole "what software" is interpreted as EC chunk - this function tries to fix that issue by
            omitting wh- word if EC candidate consists of multiple tokens, and first token in question is wh- word.
            The same issue occurs for "How" starter.
            Moreover, chunks extracted with SpaCy enclose words like 'any', 'some' which are important for us, so
            they shouldn't be substituted with 'EC' marker. Thus we remove such words if they prepend the EC.
            The result is returned as the span - the position of beginning of the fixed EC and position of end.
        """
        if (len(chunk) > 1 and
                (chunk[0].text.lower().startswith("wh")) or
                (chunk[0].text.lower() == 'how')):
            chunk = chunk[1:]

        if chunk[0].text.lower() in ['any', 'some', 'many', 'well', 'its']:
            chunk = chunk[1:]

        return (chunk.start_char, chunk.end_char)

    doc = nlp(cq)

    #  thins classified as ECs which shouldn't be interpreted that way
    rejecting_ec = ["what", "which", "when", "where", "who", "type", "types",
                    "kinds", "kind", "category", "categories", "difference",
                    "differences", "extent", "i", "we", "respect", "there",
                    "not", "the main types", "the possible types", "the types",
                    "the difference", "the differences", "the main categories"]

    counter = 1  # counter indicating current chunk id (EC1, EC2, ...)
    offset = 0   # if we replace for example "Weka" with 'EC1' then the new CQ will be shorter by one char the offset remembers by how much we have shortened the current CQ with EC markers, so the new substitutions can be applied in correct places.


    # we decided to treat qualities defined as adjectives in: How + Quality(adjective) + Verb as EC
    if (doc[0].text.lower() == 'how' and
            doc[1].pos_ == 'ADJ' and
            doc[2].pos_ == 'VERB'):

        start = doc[1].idx  # mark where quality starts
        end = start + len(doc[1])  # mark where quality ends

        cq, offset = mark_chunk(cq, [(start, end)], "EC", offset, counter)  # substitute quality with EC identifier
        counter += 1  # the next EC chunk should have a new, bigger identifier, for example EC2

    for chunk in doc.noun_chunks:  # for each EC chunk candidate detected
        (start, end) = _get_EC_span_reject_wh_starters(chunk)  # check where chunk begins and ends

        ec = cq[start - offset:end - offset]  # extract text of potential EC, apply offsets

        if ec.lower() in rejecting_ec:  # if it should be rejected - do nothing
            continue

        if "the thing" in ec and end - start > len("the thing"):
            cq = cq[:start - offset] + "EC" + str(counter) +\
                " EC" + str(counter + 1) + cq[end - offset:]
            counter += 2
            offset += (end - start) - 7
        else:
            cq, offset = mark_chunk(cq, [(start, end)], "EC", offset, counter)
            counter += 1

    if (doc[-2].pos_ == 'VERB' and doc[-3].text in ['are', 'is', 'were', 'was'] and doc[-1].text == '?') or (doc[-2].pos_ in ['ADJ', 'ADV'] and doc[-1].text == '?'):
        # if CQ ends with are/is/were/was + VERB + ? or the last token is ADJective or ADVerb, treat the
        # verb / adverb / adjective as EC
        # Which animals are endangered -> endangered is EC
        # Which animals are quick -> quick is EC
        if doc[-2].text.lower() not in rejecting_ec:
            start = doc[-2].idx
            end = start + len(doc[-2])

            cq, offset = mark_chunk(
                cq, [(start, end)], "EC", offset, counter)
            counter += 1

    return cq


def get_PCs_as_spans(cq):
    def _is_auxilary(token, chunk_token_ids):
        """
            Check if given token is an auxilary verb of detected PC.
            The auxilary verb can be in a different place than the main part
            of the PC, so pos-tag-sequence based rules don't work here.
            For example in "What system does Weka require?" - the main part
            of PC is the word 'required'. The auxilary verb 'does' is separeted
            from the main part by 'Weka' noun. Thus dependency tree is used
            to identify auxilaries.
        """
        if (token.head.i in chunk_token_ids and  # if dep-tree current token's parent (head) is somewhere inside the main part of PC represented as chunk_token_ids (sequence of numeric token identifiers)
                token.dep_ == 'aux' and  # if the dep-tree label on the edge between some word from main part of PC and current token is AUX (auxilary)
                token.i not in chunk_token_ids):  # if token is outside of detected main part of PC
            return True  # yep, it's auxilary
        else:
            return False

    def _get_span(group, doc):
        id_tags = group.split(",")
        ids = [int(id_tag.split("::")[0]) for id_tag in id_tags]
        aux = None
        for token in doc:
            if _is_auxilary(token, ids):
                aux = token

        return (doc[ids[0]].idx, doc[ids[-1]].idx + len(doc[ids[-1]]),
                aux)

    def _reject_subspans(spans):
        """
            Given list of (chunk begin index, chunk end index) spans,
            return only those spans that aren't subspans of any other span.
            For instance form list [(1,10), (2,5)], the second span
            will be rejected because it is a subspan of the first one.
        """
        filtered = []
        for i, span in enumerate(spans):
            subspan = False
            for j, other in enumerate(spans):
                if i == j:
                    continue

                if span[0] >= other[0] and span[1] <= other[1]:
                    subspan = True
                    break
            if subspan is False:
                filtered.append(span)
        return filtered

    doc = nlp(cq)

    """
        Transform CQ into a form of POS-tags with token sequence identifier.
        Each token is described with "{ID}::{POS_TAG}".
        Tokens are separated with ","
        Having that form, we can extract longest sequences of expected pos-tags
        using regexes. The extracted parts can be explored to collect identifiers
        of tokens, so we know where they are located in text.
        Ex: "Kate owns a cat" should be translated into: "1::NOUN,2::VERB,3::DET,4::NOUN"
    """
    pos_text = ",".join(
        ["{id}::{pos}".format(id=id, pos=t.pos_) for id, t in enumerate(doc)])

    regexes = [   # rules describing PCs
        r"([0-9]+::(PART|VERB),?)*([0-9]+::VERB)",
        r"([0-9]+::(PART|VERB),?)+([0-9]+::AD(J|V),)+([0-9]+::ADP)",
        r"([0-9]+::(PART|VERB),?)+([0-9]+::ADP)",
    ]

    spans = []  # list of beginnings and endings of each chunk
    for regex in regexes:  # try to extract chunks with regexes
        for m in re.finditer(regex, pos_text):
            spans.append(_get_span(m.group(), doc))  # get chunk begin and end if matched
    spans = _reject_subspans(spans)  # reject subspans
    return spans


def extract_PC_chunks(cq):
    rejecting_pc = ['is', '痴', 'are', 'was', 'do', 'does', 'did', 'were',
                    'have', 'had', 'can', 'could', 'categorise', 'regarding',
                    'is of', 'are of', 'are in', 'given', 'is there']

    offset = 0
    counter = 1

    for begin, end, aux in get_PCs_as_spans(cq):
        if cq[begin - offset:end - offset].lower() in rejecting_pc:
            continue
        spans = [(begin, end)]
        if aux:
            spans.insert(0, (aux.idx, aux.idx + len(aux)))

        cq, offset = mark_chunk(cq, spans, "PC", offset, counter)
        counter += 1
    return cq


In [ ]:
# post abstraction processing
path_to_output = os.getcwd() + "/output/output.txt"
def post_abstract(path_to_output):
    genQuestion=[] 
    splitQuestions =[]
    patterns = []
    pattern_candidates = {}
    T5Cqs_patterns = []
    with open(path_to_output, 'r') as f:
      for line in f.readlines():
        line= line.replace('?', '? ')
        line2 = line.split("?")
        line3= [p.join(' ?') for p in line2]
        line3 = [p.lstrip() for p in line3]
        genQuestion.append(line3)
    genQuestionDF= pd.DataFrame(genQuestion).T
    genQuestionDF2 = genQuestionDF.drop_duplicates(subset='questions', keep="first")
    genQuestionDF.columns=['questions']

    allList=[]
    pat_candidates={}
    for sublist in  genQuestion: 
        final = []
        for i in sublist:
            if(i==''):
                continue
            for item in i.split(','):
                pat = extract_EC_chunks(item)
                pat = extract_PC_chunks(pat)
                final.append(pat)
                pat_candidates[item]=(pat)
        allList.append(final)
    return allList, pat_candidates, genQuestionDF2

In [ ]:
# save to file for human review where necessary. you import back into the process afterwards here
def processCQs():
    absWithCQs= pd.DataFrame(pat_candidates.items(), columns=['questions', 'patterns'])
    absWithCQs = absWithCQs.iloc[:-1]
    absWithCQs.to_csv(os.getcwd() + "/output/geneCQsNov.csv", sep='|')
    #import CLaRO templates
    templates= pd.read_csv(os.getcwd() + "/CLaROv2.csv", sep = ";")
    templates.columns = 'ID', 'patterns'
    return absWithCQs, templates
    #check if present in CLaRO

def compare_patterns(absWithCQs, templates):
    check = pd.merge(absWithCQs, templates, on=['patterns'], how='left', indicator='Exist')
    generated_CQs_with_patterns = check[check['Exist'] == 'both']
    check_with_cqPatterns_only = check[check['Exist'] == 'left_only']
    check_in__templates_only = check[check['Exist'] == 'right_only']
    #extract distict ptterns found in templates 
    distinct_patterns = generated_CQs_with_patterns.drop_duplicates(subset='patterns', keep="first")
    return distinct_patterns, check_in__templates_only, check_with_cqPatterns_only, generated_CQs_with_patterns

In [ ]:
#semantic similarity to check CQs that are the same and should be dropped and also find questions that not similar
def final_output(genQuestionDF2):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    usedCorpus = list(set(list(genQuestionDF2['questions'])))
    query_Corpus =list(set(list(genQuestionDF2['questions'])))
    #Compute embedding for both lists
    corpus_embed = model.encode(query_Corpus, convert_to_tensor=True)
    covidSemanticCQs= {}
    top_k = min(5, len(usedCorpus))
    for qry in query_Corpus:
        query_embed = model.encode(qry, convert_to_tensor=True, device=device)
        cos_scores = util.cos_sim(query_embed, corpus_embed)[0]
        top_results = torch.topk(cos_scores, k=top_k)
    for score, idx in zip(top_results[0], top_results[1]):
        new_score= "{:.4f}".format(score)
        new_score= float(new_score)
        if  new_score <= 0.7500:
            covidSemanticCQs[qry]=[]
            covidSemanticCQs[qry].append({query_Corpus[idx]: (new_score)})
        QuesSimilar= pd.json_normalize([ { "firstCQs": key_, "secondCQs" : i, "score" : child[i] }  for key_ in covidSemanticCQs for child in covidSemanticCQs[key_] for i in child ])

    QuesSimilar.to_csv(os.getcwd() +  "/output/semanticCQs.csv", sep='|')
    return QuesSimilar

In [ ]:
#generate CQs      
def generateCQs(textFile):   
    outText= readTextFile(textFile)
    # text = format(outText) I need to add outText to process file (stuck here)
    preprocessor= TextPreprocessor()
    processed_text = preprocessor.process_file(text=outText, output_path=None,
                                               remove_numbers=True,
                                               remove_headers=True,
                                               normalize_spaces=True,
                                               fix_breaks=True,
                                               clean_chars=True,
                                               merge_sentences=True)
    processor = SentenceProcessor()
    sentences, corpus_entities = processor.process_text(processed_text)
    # get only the sentences for process_test output and generate dataframe
    df= pd.DataFrame([sentences])
#   rename column
    df.columns = ['sentences']
    df['sentences_new'] = df['sentences'].map(lambda x: ' '.join(x))
    print("a look at dataframe",df.head(10))

    cqs = generate(df)
    dcqs = postprocess(cqs)
    allList, pat_candidates, genQuestionDF2 = post_abstract(path_to_output)
    Similar_CQs_filter= final_output(genQuestionDF2)
    absWithCQs,templates= processCQs(pat_candidates)
    distinct_patterns, check_in_templates_only, check_with_cqPatterns_only, generated_CQs_with_patterns = compare_patterns(absWithCQs, templates)
        
    return Similar_CQs_filter, distinct_patterns, generated_CQs_with_patterns, corpus_entities
    
def main(file_paths):
    list_text_files = []
    text_file_path= get_files(file_paths)
    duplicate_CQs = []
    duplicate_CQs_df = pd.DataFrame(duplicate_CQs)
    files_used =pd.DataFrame(list_text_files)
    for textFile in text_file_path:
        if textFile.endswith(".txt"):
            CQs_file_name = Path(textFile).stem
            result_name = CQs_file_name.join("_CQs")
            Similar_CQs_filter, distinct_patterns, result_name  = generateCQs(textFile)
            duplicate_CQs.append(Similar_CQs_filter)
            list_text_files.append(CQs_file_name)

    # save outputs to file

    return list_text_files, Similar_CQs_filter, distinct_patterns, result_name 

if __name__ == "__main__":
    list_text_files, Similar_CQs_filter, distinct_patterns, result_name = main(file_paths)  


        